In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Module 3.1- Run LandTrendr over Composites

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/redcastle-resources/lcms-training/blob/main/3.1-LandTrendr.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/redcastle-resources/lcms-training/blob/main/3.1-LandTrendr.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://github.com/redcastle-resources/lcms-training/blob/main/3.1-LandTrendr.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview

LandTrendr, short for Landsat-based detection of Trends in Disturbance and Recovery, is a temporal segmentation algorithm used for change detection as well as time-series smoothing. You can use LandTrendr outputs for stand-alone change detection or as inputs to modeling models such as the ones used by LCMS. 

**This notebook uses Landsat and Sentinel-2 composites, like the ones you created in the previous notebook, as inputs to the LandTrendr temporal segmentation and spectral smoothing operations.**

##### To learn more about LandTrendr:
- [LandTrendr Guide](https://emapr.github.io/LT-GEE/)
- [LandTrendr Original Publication](https://www.sciencedirect.com/science/article/abs/pii/S0034425710002245)
- [LandTrendr GEE Publication](https://www.mdpi.com/2072-4292/10/5/691)


### Objective

In this tutorial, you learn how to create and manipulate LandTrendr outputs.

**This tutorial uses the following Google Cloud services:**

- Google Earth Engine

**The steps performed include:**

- Create LandTrendr outputs
- Manipulate EE array image objects to get meaningful data out of LandTrendr outputs

**Learning objectives include:**
- Users will understand the purpose of temporal segmentation as implemented in LandTrendr.
- Users will understand key parameters in LandTrendr algorithm. 
- Users will be able to generate and manipulate array outputs from LandTrendr. 


## Before you begin

### Set your current URL under `workbench_url`
This gives the Map Viewer a url in which to host the viewer we will be generating. 
* This will be in your URL/search bar at the top of the browser window you are currently in
* It will look something like `https://1234567890122-dot-us-west3.notebooks.googleusercontent.com/` (See the image below)

![workspace url](img/workspace-url.png)

### Set a folder to use for all exports under `export_path_root` 
* This folder should be an assets folder in an existing GEE project.
* By default, this folder is the same as the pre-baked folder (where outputs have already been created). 
* If you would like to create your own outputs, specify a different path for `export_path_root`, but leave the `pre_baked_path_root` as it was. This way, the pre-baked outputs can be shown at the end, instead of waiting for all exports to finish.
* It will be something like `projects/projectID/assets/someFolder`
* This folder does not have to already exist. If it does not exist, it will be created

In [1]:
workbench_url = 'https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com/'
pre_baked_path_root  = 'projects/rcr-gee/assets/lcms-training'
export_path_root = pre_baked_path_root
print('Done')

Done


### Installation
First, install necessary Python packages. Uncomment the first line to upgrade geeViz if necessary.

In [2]:
#Module imports
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
import geeViz.gee2Pandas as g2p

import inspect

ee = getImagesLib.ee
Map = getImagesLib.Map

print('Done')

Initializing GEE
Successfully initialized
geeViz package folder: /opt/conda/lib/python3.10/site-packages/geeViz
Done


### Set up your work environment

Specify an image collection path where composites were exported to. In addition, create a blank image collection where your LandTrendr outputs will be exported to.

Currently, when running within Colab or Workbench, geeView uses a different project to authenticate through, so you may need to make your asset public to view from within Colab.

In [4]:
# Create folder and a collection and make public

export_composite_collection = f'{export_path_root}/lcms-training_module-2_composites'
export_landTrendr_collection = f'{export_path_root}/lcms-training_module-3_landTrendr'

aml.create_asset(export_landTrendr_collection,asset_type = ee.data.ASSET_TYPE_IMAGE_COLL)

# Currently geeView within Colab uses a different project to authenticate through, so you may need to make your asset public to view from within Colab
aml.updateACL(export_landTrendr_collection,writers = [],all_users_can_read = True,readers = [])

print('Done')

Found the following sub directories:  ['lcms-training', 'lcms-training_module-3_landTrendr']
Will attempt to create them if they do not exist
Asset projects/rcr-gee/assets/lcms-training already exists
Asset projects/rcr-gee/assets/lcms-training/lcms-training_module-3_landTrendr already exists
Updating permissions for:  projects/rcr-gee/assets/lcms-training/lcms-training_module-3_landTrendr
Done


## Change detection with LandTrendr - intro

LandTrendr is a temporal segmentation algorithm developed at Oregon State University. 

LandTrendr works by taking a time series of annual values and fitting multiple linear regression models to recursively break the time series into segments that represent time periods with similar linear trends. The resulting segments can be used to describe change processes on the landscape. 

![workspace url](https://emapr.github.io/LT-GEE/imgs/segmentation.png)

By having the duration, magnitude, slope, etc of each segment, you can more easily detect change with simple rulesets.

![workspace url](https://emapr.github.io/LT-GEE/imgs/segment_attributes.png)
For example, a short, steep decline is likely to be a fast change, like harvest or fire. A long, shallow decline is likely to be a slower change, due to insects, disease, or drought. LandTrendr can also track periods of recovery, like a long shallow incline following a harvest or fire event. 



### Example 
View an example of data input to LandTrendr data -- don't worry about the params too much yet. 

In [5]:
# set up map
Map.clearMap()

# reset port if necessary
Map.port = 1235
Map.proxy_url = workbench_url

#### LandTrendr bands and parameters

Lets look at the bands included in the **actual** collection that LCMS uses for LandTrendr in PuertoRico

In [6]:
lcms_actual_lt_collection = ee.ImageCollection('projects/lcms-292214/assets/R8/PR_USVI/Base-Learners/LandTrendr-Collection-1984-2020')
print('All bands LCMS uses for LandTrendr:',lcms_actual_lt_collection.aggregate_histogram('band').keys().getInfo())

All bands LCMS uses for LandTrendr: ['NBR', 'NDMI', 'NDSI', 'NDVI', 'blue', 'brightness', 'green', 'greenness', 'nir', 'red', 'swir1', 'swir2', 'tcAngleBG', 'wetness']


#### Composites
Inspect composites that we're bringing in, get info, calculate additional indices to use for LandTrendr

In [9]:
# Bring in composites and pull info from them
composites = ee.ImageCollection(export_composite_collection)

props = composites.first().toDictionary().getInfo()

startYear = props['startYear']
endYear = props['endYear']

startJulian = props['startJulian']
endJulian = props['endJulian']

proj = composites.first().projection().getInfo()

# Pull out the crs
# Depending on if a wkt or epsg format is used, it will be stored under a different key
if 'crs' not in proj.keys():
    crs = proj['wkt']
else:
    crs = proj['crs']
    
transform = proj['transform']
scale = None

studyArea = composites.first().geometry()

# Decompress composites by dividing by 10000 for optical bands and add indices
composites = composites.select(['blue','green','red','nir','swir1','swir2']).map(lambda img: img.divide(10000).float().copyProperties(img,['system:time_start']))
composites = composites.map(getImagesLib.simpleAddIndices)\
                      .map(getImagesLib.getTasseledCap)\
                      .map(getImagesLib.simpleAddTCAngles)


print('Done')

Done


In [20]:
# First let's explore the composites and some attributes that can help understand how well the composites turned out
Map.clearMap()
Map.addTimeLapse(composites,getImagesLib.vizParamsFalse,'Composites')

Map.centerObject(studyArea,9)
Map.turnOnInspector()
Map.view()

Adding layer: Composites
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


127.0.0.1 - - [23/Aug/2023 15:51:20] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2023 15:51:24] "GET /geeView/images/cumulative_icon.png HTTP/1.1" 200 -


### Set parameters for LandTrendr

We can use any/all of the bands, but generally bands that use nir and swir are most useful


## LandTrendr Params
| Argument               | Type                    | Details                                                                                                                                                           |
|------------------------|-------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| timeSeries             | ImageCollection         | Yearly time-series from which to extract breakpoints. The first band is usedto find breakpoints, and all subsequent bands are fitted using those breakpoints.     |
| maxSegments            | Integer                 | Maximum number of segments to be fitted on the time series.                                                                                                       |
| spikeThreshold         | Float, default: 0.9     | Threshold for dampening the spikes (1.0 means no dampening).                                                                                                      |
| vertexCountOvershoot   | Integer, default: 3     | The initial model can overshoot the maxSegments + 1 vertices by this amount. Later, it will be pruned down to maxSegments + 1.                                    |
| preventOneYearRecovery | Boolean, default: False | Prevent segments that represent one year recoveries.                                                                                                              |
| recoveryThreshold      | Float, default: 0.25    | If a segment has a recovery rate faster than 1/recoveryThreshold (in years), then the segment is disallowed.                                                      |
| pvalThreshold          | Float, default: 0.1     | If the p-value of the fitted model exceeds this threshold, then the current model is discarded and another one is fitted using the Levenberg-Marquardt optimizer. |
| bestModelProportion    | Float, default: 0.75    | Allows models with more vertices to be chosen if their p-value is no more than (2 - bestModelProportion) times the p-value of the best model.                     |
| minObservationsNeeded  | Integer, default: 6     | Min observations needed to perform output fitting.                                                                                                                |


We generally use the default parameters from the Kennedy et al. 2018 paper (Table 1 below) 

|       Parameter       |  IDL |   GEE   |                                                                                                        Comments                                                                                                       |
|:---------------------:|:----:|:-------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
|      maxSegments      |   6  |    6    |                                                                                                                                                                                                                       |
|     spikeThreshold    |  0.9 |   0.9   | Renamed from “desawtooth val”                                                                                                                                                                                         |
|  vertexCountOvershoot |   3  |    3    |                                                                                                                                                                                                                       |
|   recoveryThreshold   | 0.25 |   0.25  |                                                                                                                                                                                                                       |
|     pvalThreshold     | 0.05 |   0.05  |                                                                                                                                                                                                                       |
|  bestModelProportion  | 0.75 |   0.75  |                                                                                                                                                                                                                       |
| minObservationsNeeded |   6  |    6    | Renamed from “minneeded”                                                                                                                                                                                              |
|     Background_val    |   0  |    NA   | GEE uses a mask logic to avoid missing values caused by clouds, shadows, and missing imagery.                                                                                                                         |
|        Divisor        |  −1  |    NA   | Ensures that vegetation loss disturbance results in negative change in value when NBR is used as a spectral metric. In GEE, this must be handled outside of the segmentation algorithm.                               |
|       Kernelsize      |   1  | Dropped | Originally used together with skipfactor to save computational burden; no longer necessary.                                                                                                                           |
|       Skipfactor      |   1  | Dropped |                                                                                                                                                                                                                       |
|    Distweightfactor   |   2  | Dropped | Inadvertently hardwired in the IDL code, this parameter was hardwired in the GEE code to the value of 2.                                                                                                              |
|     Fix_doy_effect    |   1  | Dropped | Although correcting day-of-year trends was considered theoretically useful in the original LT implementation, in practice it has been found to distort time series values when change occurs and thus was eliminated. |
Run the code block below to set params

In [7]:
# We can use any/all of the bands, but generally bands that use nir and swir are most useful
bandNames = ['red','nir','swir1','swir2','NBR','NDVI','brightness','greenness','wetness']

#Define landtrendr params
run_params = { 
  'maxSegments':            6,
  'spikeThreshold':         0.9,
  'vertexCountOvershoot':   3,
  'preventOneYearRecovery': False,
  'recoveryThreshold':      0.25,
  'pvalThreshold':          0.05,
  'bestModelProportion':    0.75,
  'minObservationsNeeded':  6
}

print('Done')

Done


### Run LandTrendr - on NBR only 
NBR is the Normmalized Burn Ratio. It is sensitive to changes in moisture and/or vegetation cover

Now let's run LandTrendr over NBR only, as a test example that will allow us to look at the outputs


In [11]:
Map.clearMap()

test_band = 'NBR'
run_params['timeSeries'] = composites.select([test_band])
raw_LT = ee.Algorithms.TemporalSegmentation.LandTrendr(**run_params)

Map.addLayer(raw_LT,{},'LT Raw {}'.format(test_band),True)

Map.turnOnInspector()
Map.view()

Adding layer: LT Raw NBR
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Starting local web server at: http://localhost:8001/geeView/
HTTP server command: "/opt/conda/bin/python" -m http.server  8001
Done
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


127.0.0.1 - - [23/Aug/2023 17:55:54] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


Double click on the output on the map to query it. 

Notice this output is not immediately useful for change detection or smoothing out a time series of composites


### Using LandTrendr for Change Detection

* We will now go through each step to take the raw LandTrendr output and create a basic change detection output
* An single pixel will be used to show the impact each step has on the array that is being manipulated


In [26]:
# Provide an example location 
pt = ee.Geometry.Point([ -65.944 , 18.404])

 
# First, we will select the LandTrendr image array output band
lt_array = raw_LT.select(['LandTrendr'])

display(g2p.imageArrayPixelToDataFrame(lt_array, pt,None,crs,transform, 'Complete Raw LandTrendr Output',\
                                       ['Years','Raw Input Values','Fitted Output Values','Vertex/non-vertex']))
# This output has 2 dimensions per pixel
# The rows correspond to 0-years, 1-raw input values, 2-LandTrendr fitted output values, and 3-vertex/non vertex

# First, slice the vertices  and use them as a mask to mask out non vertex values
vertices = lt_array.arraySlice(0,3,4)
display(g2p.imageArrayPixelToDataFrame(vertices, pt, None,crs,transform,'Only the vertex mask row'))
lt_array = lt_array.arrayMask(vertices)
display(g2p.imageArrayPixelToDataFrame(lt_array, pt, None,crs,transform,'Complete Raw LandTrendr Only Vertex Columns',\
                                                 ['Years','Raw Input Values','Fitted Output Values','Vertex/non-vertex']))

# In order to perform change detection, we'll need to get the difference between fitted vertex values
# We do this by slicing the first to the next to the last and then the second to the last and subtracting them
left = lt_array.arraySlice(1,0,-1)
right = lt_array.arraySlice(1,1,None)
diff  = left.subtract(right)

display(g2p.imageArrayPixelToDataFrame(left, pt, None,crs,transform,'Left Slice',\
                                       ['Years','Raw Input Values','Fitted Output Values','Vertex/non-vertex']))
display(g2p.imageArrayPixelToDataFrame(right, pt,None,crs,transform, \
                                       'Right Slice',['Years','Raw Input Values','Fitted Output Values','Vertex/non-vertex']))
display(g2p.imageArrayPixelToDataFrame(diff, pt, None,crs,transform,\
                                       'Left Minus Right',['Years','Raw Input Values','Fitted Output Values','Vertex/non-vertex']))


# We then slice the right-hand years and the fitted vertex values difference and combine them
years = right.arraySlice(0,0,1)
mag = diff.arraySlice(0,2,3)
forSorting = years.arrayCat(mag,0)
display(g2p.imageArrayPixelToDataFrame(forSorting, pt, None,crs,transform,'Array Ready For Sorting'))

# We can then sort using various rows depending on whether we want the highest magnitude change, most recent, etc
# In this example, the sort row will be the magnitude, thus giving us the highest severity change
sorted = forSorting.arraySort(forSorting.arraySlice(0,1,2))
display(g2p.imageArrayPixelToDataFrame(sorted, pt, None,crs,transform,'Array Sorted by the Second Row (magnitude of change)'))


# Slice off the year and magnitude of the highest magnitude change
highest_mag_change_array = sorted.arraySlice(1,0,1)
display(g2p.imageArrayPixelToDataFrame(highest_mag_change_array, pt, None,crs,transform,'Highest Mag Change (year and magnitude)'))


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
Years,1984.000000,1985.000000,1986.000000,1987.000000,1988.000000,1989.000000,1990.000000,1992.000000,1999.000000,2000.000000,2002.000000,2003.000000,2006.000000,2009.000000,2010.000000,2012.000000,2013.000000,2014.000000,2015.000000,2016.000000,2017.000000,2018.000000,2019.000000,2020.000000,2021.000000,2022.000000
Raw Input Values,0.732861,0.567021,0.772094,0.716129,0.765138,0.656260,0.674274,0.714593,0.100228,0.086324,0.098278,0.035483,0.148884,0.076102,0.068394,0.125019,0.277041,0.380944,0.452543,0.547757,0.615267,0.552308,0.558119,0.336950,0.426205,0.430944
Fitted Output Values,0.759414,0.730053,0.700691,0.671330,0.641969,0.612607,0.583246,0.524523,0.318994,0.289633,0.230910,0.201549,0.113465,0.025381,-0.003980,-0.062703,0.111118,0.284938,0.458759,0.632580,0.589873,0.547167,0.504460,0.461754,0.419048,0.376341
Vertex/non-vertex,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1


,0,1,2,3
Years,1984.000000,2012.000000,2016.000000,2022.000000
Raw Input Values,0.732861,0.125019,0.547757,0.430944
Fitted Output Values,0.759414,-0.062703,0.632580,0.376341
Vertex/non-vertex,1.000000,1.000000,1.000000,1.000000


,0,1,2
Years,1984.000000,2012.000000,2016.000000
Raw Input Values,0.732861,0.125019,0.547757
Fitted Output Values,0.759414,-0.062703,0.632580
Vertex/non-vertex,1.000000,1.000000,1.000000


,0,1,2
Years,2012.000000,2016.000000,2022.000000
Raw Input Values,0.125019,0.547757,0.430944
Fitted Output Values,-0.062703,0.632580,0.376341
Vertex/non-vertex,1.000000,1.000000,1.000000


,0,1,2
Years,-28.000000,-4.000000,-6.000000
Raw Input Values,0.607841,-0.422738,0.116814
Fitted Output Values,0.822117,-0.695283,0.256238
Vertex/non-vertex,0.000000,0.000000,0.000000


,0,1,2
0,2012.000000,2016.000000,2022.000000
1,0.822117,-0.695283,0.256238


,0,1,2
0,2016.000000,2022.000000,2012.000000
1,-0.695283,0.256238,0.822117


,0
0,2016.000000
1,-0.695283


### Convert arrays into images
But those arrays still aren't immediately usable

Arrays will be converted into images representing loss magnitude and loss year

In [28]:
# First, choose a change threshold
# Any changes more negative than this value will be flagged as loss
change_threshold = -0.15

In [29]:
# Convert the sorted array image into a 2-band image
highest_mag_change = highest_mag_change_array.arrayProject([0]).arrayFlatten([['yr','mag']])

# Mask out any change 
highest_mag_change = highest_mag_change.updateMask(highest_mag_change.select(['mag']).lte(change_threshold))

# Pull the los magnitude palette and flip the color order
lossMagPalette = changeDetectionLib.lossMagPalette.split(',')
lossMagPalette.reverse()

# Set up map
Map.clearMap()
Map.addLayer(forSorting,{},'Array for Sorting',False)
Map.addLayer(sorted,{},'Sorted Array',False)
Map.addLayer(highest_mag_change.select(['mag']),{'min':-0.8,'max':-0.15,'palette':lossMagPalette},'Loss Magnitude')
Map.addLayer(highest_mag_change.select(['yr']),{'min':startYear,'max':endYear,'palette':changeDetectionLib.lossYearPalette},'Loss Year')
Map.turnOnInspector()
Map.view()

Adding layer: Array for Sorting
Adding layer: Sorted Array
Adding layer: Loss Magnitude
Adding layer: Loss Year
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


127.0.0.1 - - [23/Aug/2023 18:05:17] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


## Run full LandTrendr - on all bands

Explain what each section of the code below is doing 
i.e., running LandTrendr for all bands and processing the outut into images that represent changes of interest

In [30]:
# We will run LandTrendr for each band
#Clear the map in case it has been populated with layers/commands earlier
Map.clearMap()

# We can use any/all of the bands, but generally bands that use nir and swir are most useful
bandNames = ['red','nir','swir1','swir2','NBR','NDVI','brightness','greenness','wetness']

#Run LANDTRENDR
for bandName in bandNames:

    # Select the band and run LandTrendr
    run_params['timeSeries'] = composites.select([bandName])
    rawLT = ee.Algorithms.TemporalSegmentation.LandTrendr(**run_params)

    Map.addLayer(rawLT,{},'LT Raw {}'.format(bandName),False)

    # Notice the raw LandTrendr output is in GEE's image array format
    # We'll need to manipulate the raw output a bit to save on storage space

    # Mask out non vertex values to use less storage space
    ltArray = rawLT.select(['LandTrendr'])
    rmse = rawLT.select(['rmse'])
    vertices = ltArray.arraySlice(0,3,4)
    ltArray = ltArray.arrayMask(vertices)

    # Mask out all but the year and vertex fited values (get rid of the raw and vertex rows)
    ltArray = ltArray.arrayMask(ee.Image(ee.Array([[1],[0],[1],[0]])))
    rawLTForExport=ltArray.addBands(rmse)
    Map.addLayer(rawLTForExport,{},'LT Vertex Values Only {}'.format(bandName),False)

    # Show how the compressed vertex-only values can be decompressed later
    decompressedC = changeDetectionLib.simpleLTFit(ltArray,startYear,endYear,bandName,True,run_params['maxSegments'])
    Map.addLayer(decompressedC,{'bands':'{}_LT_fitted'.format(bandName),'min':0.2,'max':0.8},'Decompressed LT Output {}'.format(bandName),False)

    # Join the raw and fitted values
    fitted = decompressedC.select(['{}_LT_fitted'.format(bandName)])
    ltJoined = getImagesLib.joinCollections(composites.select([bandName]),fitted)
    Map.addLayer(ltJoined,{'bands':'{}_LT_fitted'.format(bandName),'min':0.2,'max':1,'palette':'D80,080'},'Raw and LT Fitted {}'.format(bandName),True)

    # Export LT array image
    # Set some properties that will be uses later
    rawLTForExport = rawLTForExport.set({'startYear':startYear,
                                          'endYear':endYear,
                                          'startJulian':startJulian,
                                          'endJulian':endJulian,
                                          'band':bandName})
    rawLTForExport =rawLTForExport.set(run_params)
    exportName = 'LT_Raw_{}_yrs{}-{}_jds{}-{}'.format(bandName,startYear,endYear,startJulian,endJulian)
    exportPath = export_landTrendr_collection + '/'+ exportName
    # Export output
    getImagesLib.exportToAssetWrapper(rawLTForExport,exportName,exportPath,{'.default':'sample'},studyArea,scale,crs,transform,overwrite=False)

Map.turnOnInspector()
Map.addLayer(studyArea, {'strokeColor': '0000FF'}, "Study Area", False)
Map.view()

Adding layer: LT Raw red
Adding layer: LT Vertex Values Only red
Adding layer: Decompressed LT Output red
Adding layer: Raw and LT Fitted red
LT_Raw_red_yrs1984-2022_jds152-151 currently exists or is being exported and overwrite = False. Set overwite = True if you would like to overwite any existing asset or asset exporting task
Adding layer: LT Raw nir
Adding layer: LT Vertex Values Only nir
Adding layer: Decompressed LT Output nir
Adding layer: Raw and LT Fitted nir
LT_Raw_nir_yrs1984-2022_jds152-151 currently exists or is being exported and overwrite = False. Set overwite = True if you would like to overwite any existing asset or asset exporting task
Adding layer: LT Raw swir1
Adding layer: LT Vertex Values Only swir1
Adding layer: Decompressed LT Output swir1
Adding layer: Raw and LT Fitted swir1
LT_Raw_swir1_yrs1984-2022_jds152-151 currently exists or is being exported and overwrite = False. Set overwite = True if you would like to overwite any existing asset or asset exporting ta

127.0.0.1 - - [23/Aug/2023 18:05:41] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


In [25]:
# Can track tasks here or at https://code.earthengine.google.com/tasks
# If you'd like to track the tasks, use this:
# tml.trackTasks2()

# If you want to cancel all running tasks, you can use this function
# tml.batchCancel()

# If you want to empty the collection of all images
# aml.batchDelete(export_landTrendr_collection, type = 'imageCollection')

print('done')

done


Array processing also happens within functions 

`changeDetectionLib.convertToLossGain()` - https://github.com/gee-community/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/changeDetectionLib.py#L778C5-L778C22



### Convert LandTrendr array into time series - for input into LCMS

https://github.com/gee-community/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/changeDetectionLib.py#L565




In [31]:
# While we can use the LandTrendr output for change detection, LCMS uses it as inputs to change detection, land cover, and land use models
# This will show how the raw LandTrendr array image asset is converted into a time series 
# of annual fitted, segment duration, segment magnitude of change, and slope values
Map.clearMap()

lt_asset = ee.ImageCollection(f'{pre_baked_path_root}/lcms-training_module-3_landTrendr')
# Convert stacked outputs into collection of fitted, magnitude, slope, duration, etc values for each year
# While the fitted LandTrendr value is generally of most importance to our models, 
# LandTrendr segment duration, slope, and magnitude of change can also help our models
lt_fit = changeDetectionLib.batchSimpleLTFit(lt_asset,startYear,endYear,None,bandPropertyName='band',arrayMode=True)

# Vizualize image collection that is used as predictors in LCMS models
# When you double-click the output on the map, notice the different values available to the models
Map.addLayer(lt_fit,{'bands':'swir2_LT_fitted,nir_LT_fitted,red_LT_fitted','min':0.15,'max':0.6},'LandTrendr All Predictors Time Series')


Map.turnOnInspector()
Map.addLayer(studyArea, {'strokeColor': '0000FF'}, "Study Area", False)
Map.view()

Adding layer: LandTrendr All Predictors Time Series
Adding layer: Study Area
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


127.0.0.1 - - [23/Aug/2023 18:07:55] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


#### Inspect
Double-click the map to query values and see actual and fitted values for indices over time

#### LandTrendr reduces noise in original composite tiem series 

The best way of understanding how LandTrendr contributes to reducing noise in the original composite time series is to visualize them side-by-side
This example takes the fitted values from LandTrendr and shows them along with the original
Notice many holes are now filled in by LandTrendr
In general the amount of noise is reduced. There is a risk however of fitting too much and omitting changes such as those seen in 2017 for Hurricane Maria

INCLUDE A SCREENSHOT OF HURRICANE MARIA

Click on the map, inspect the fitted time series, watch the composite timelapse and the Landtrendr timelapse

In [32]:
Map.clearMap()

# The best way of understanding how LandTrendr contributes to reducing noise in the original composite time series
# is to visualize them side-by-side
# This example takes the fitted values from LandTrendr and shows them along with the original
# Notice many holes are now filled in by LandTrendr
# In general the amount of noise is reduced. There is a risk however of 
# fitting too much and omitting changes such as those seen in 2017 for Hurricane Maria

# Visualize fitted landTrendr composites
fitted_bns = lt_fit.select(['.*_fitted']).first().bandNames()
out_bns = fitted_bns.map(lambda bn: ee.String(bn).split('_').get(0))

# Give same names as composites
lt_synth = lt_fit.select(fitted_bns,out_bns)

# Visualize raw and LandTrendr fitted composites
Map.addTimeLapse(composites,getImagesLib.vizParamsFalse,'Raw Composite Timelapse')

Map.addTimeLapse(lt_synth,getImagesLib.vizParamsFalse,'Fitted LandTrendr Composite Timelapse')


# Join the raw and fited values
ltJoined = getImagesLib.joinCollections(composites.select(bandNames),lt_fit.select(['.*_fitted']))

Map.addLayer(ltJoined,{'min':0.2,'max':1},'Raw and LT Fitted',True)


Map.turnOnInspector()
Map.addLayer(studyArea, {'strokeColor': '0000FF'}, "Study Area", False)
Map.view()

Adding layer: Raw Composite Timelapse
Adding layer: Fitted LandTrendr Composite Timelapse
Adding layer: Raw and LT Fitted
Adding layer: Study Area
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:8001/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://53c21733d8125e22-dot-us-west3.notebooks.googleusercontent.com/proxy/8001/geeView/?accessToken=None


127.0.0.1 - - [23/Aug/2023 18:08:39] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


### Vizualize other LandTrendr data

-- pull in example from https://github.com/redcastle-resources/geeViz/blob/master/examples/LANDTRENDRViz.py -- to show all LandTrendr outputs at the same time 

### Challenge: 

Calculate LandTrendr stats for NDVI and process arrays into most recent change year 

### Congratulations! You're done with module 3.1 

Other GeeViz LandTrendr examples: 
- https://github.com/redcastle-resources/geeViz/blob/master/examples/LANDTRENDRViz.py
- https://github.com/redcastle-resources/geeViz/blob/master/examples/LANDTRENDRWrapper.py
- https://github.com/redcastle-resources/geeViz/blob/master/examples/LANDTRENDRWrapperNotebook.ipynb


LandTrendr fitted data will be used as inputs to LCMS in subsequent modules